In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10,mnist,cifar100
from keras import Sequential,optimizers
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import LoggerYN as YN
import numpy as np
import scipy.io as sio
import utilsYN as uYN
import datetime
import time

In [ ]:
def initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    global Dataset    
    global pbatchSize
    global pnumClasses
    global pEpochs
    global pLearningRate
    global pMomentum
    global pWeightDecay
    Dataset = dataset
    pbatchSize = batchSize
    pnumClasses = numClasses
    pEpochs = epochs
    pLearningRate = learningRate
    pMomentum = momentum
    pWeightDecay = weightDecay
    
def NormalizeData(x_train,x_test):
        x_train /= 255
        x_test /= 255
        return x_train, x_test

def CategorizeData(y_train,y_test,pnumClasses):
    y_train = keras.utils.to_categorical(y_train, pnumClasses)
    y_test = keras.utils.to_categorical(y_test, pnumClasses)
    
    
    return y_train, y_test
    
def loadData():
    fineFlag= False
    global Dataset
    if Dataset == "mnist":
        Dataset = mnist
    elif Dataset == "cifar100":
        Dataset = cifar100
    else:
        Dataset = cifar10
    (x_train, y_train), (x_test, y_test) = Dataset.load_data(label_mode='fine') if fineFlag else Dataset.load_data()
    
    global imgRows
    global imgCols
    global inputShape
    
    imgRows = x_train.shape[1]
    imgCols = x_train.shape[2]

    try:
        imgRGB_Dimensions = x_train.shape[3]
    except Exception:
        imgRGB_Dimensions = 1 #For Gray Scale Images

    print(x_train.shape)
    x_train = x_train.reshape(x_train.shape[0], imgRows, imgCols, imgRGB_Dimensions)
    x_test = x_test.reshape(x_test.shape[0], imgRows, imgCols, imgRGB_Dimensions)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train, x_test = NormalizeData(x_train, x_test)
    y_train, y_test = CategorizeData(y_train,y_test,pnumClasses)
    inputShape = (imgRows, imgCols, imgRGB_Dimensions)
    return x_train, y_train, x_test, y_test


def loadDataSVHN(fname = './%s_32x32.mat',extra=False):
    """Load the SVHN dataset (optionally with extra images)
    Args:
        extra (bool, optional): load extra training data
    Returns:
        Dataset: SVHN data
    """
    def load_mat(fname):
        data = sio.loadmat(fname)
        X = data['X'].transpose(3, 0, 1, 2)
        y = data['y'] % 10  # map label "10" --> "0"
        return X, y

    data = uYN.Dataset()
    data.classes = np.arange(10)

    X, y = load_mat(fname % 'train')
    data.train_images = X
    data.train_labels = y.reshape(-1)

    X, y = load_mat(fname % 'test')
    data.test_images = X
    data.test_labels = y.reshape(-1)

    new_x = data.train_images
    new_y = data.train_labels
    
    if extra:
        X, y = load_mat(fname % 'extra')
        data.extra_images = X
        data.extra_labels = y.reshape(-1)
    
        # Use extra dataset
        new_x = data.extra_images#np.concatenate((data.extra_images, data.train_images), axis=0)
        new_y = data.extra_labels#np.concatenate((data.extra_labels, data.train_labels), axis=0)

    (x_train, y_train), (x_test, y_test)  = (new_x,new_y),(data.test_images,data.test_labels)
    
    global imgRows
    global imgCols
    global inputShape
    
    imgRows = x_train.shape[1]
    imgCols = x_train.shape[2]

    try:
        imgRGB_Dimensions = x_train.shape[3]
    except Exception:
        imgRGB_Dimensions = 1 #For Gray Scale Images

    print(x_train.shape)
    
    x_train = x_train.reshape(x_train.shape[0], imgRows, imgCols, imgRGB_Dimensions)
    x_test = x_test.reshape(x_test.shape[0], imgRows, imgCols, imgRGB_Dimensions)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train, x_test = NormalizeData(x_train, x_test)
    y_train, y_test = CategorizeData(y_train,y_test,10)
    inputShape = (imgRows, imgCols, imgRGB_Dimensions)
    return x_train, y_train, x_test, y_test


def model_MNIST():
    MNIST_model = Sequential()
    MNIST_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=inputShape))
    MNIST_model.add(Conv2D(64, (3, 3), activation='relu'))
    MNIST_model.add(MaxPooling2D(pool_size=(2, 2)))
    MNIST_model.add(Dropout(0.25))
    MNIST_model.add(Flatten())
    MNIST_model.add(Dense(128, activation='relu'))
    MNIST_model.add(Dropout(0.5))
    MNIST_model.add(Dense(pnumClasses, activation='softmax'))     
    return MNIST_model

def model_CIFAR10():
    CIFAR_model = Sequential()
    CIFAR_model.add(Conv2D(32, (3, 3), padding='same',input_shape=inputShape))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(Conv2D(32, (3, 3)))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(MaxPooling2D(pool_size=(2, 2)))
    CIFAR_model.add(Dropout(0.25))
    CIFAR_model.add(Conv2D(64, (3, 3), padding='same'))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(Conv2D(64, (3, 3)))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(MaxPooling2D(pool_size=(2, 2)))
    CIFAR_model.add(Dropout(0.25))
    CIFAR_model.add(Flatten())
    CIFAR_model.add(Dense(512))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(Dropout(0.5))
    CIFAR_model.add(Dense(pnumClasses))
    CIFAR_model.add(Activation('softmax'))    
    return CIFAR_model

def model_CIFAR100():
    CIFAR_model = Sequential()
    CIFAR_model.add(Conv2D(128, (3, 3), padding='same',strides=1,input_shape=inputShape))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(Conv2D(128, (3, 3)))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(MaxPooling2D(pool_size=(2, 2),strides=2,padding='valid'))
    CIFAR_model.add(Dropout(0.1))
    
    CIFAR_model.add(Conv2D(256, (3, 3), padding='same',strides=1))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(Conv2D(256, (3, 3)))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(MaxPooling2D(pool_size=(2, 2),strides=2,padding='valid'))
    CIFAR_model.add(Dropout(0.25))
    
    CIFAR_model.add(Conv2D(512, (3, 3), padding='same',strides=1))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(Conv2D(512, (3, 3)))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(MaxPooling2D(pool_size=(2, 2),strides=2,padding='valid'))
    CIFAR_model.add(Dropout(0.5))
    
    
    CIFAR_model.add(Flatten())
    CIFAR_model.add(Dense(1024))
    CIFAR_model.add(Activation('relu'))
    CIFAR_model.add(Dropout(0.5))
    CIFAR_model.add(Dense(pnumClasses))
    CIFAR_model.add(Activation('softmax'))    
    return CIFAR_model

def model_SVHC():
    model_SVHC = Sequential()
    model_SVHC.add(Conv2D(48, (5, 5), padding='same',input_shape=inputShape))
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=2))
    model_SVHC.add(Dropout(0.2))
    
    model_SVHC.add(Conv2D(64, (5, 5), padding='same'))
    model_SVHC.add(keras.layers.normalization.BatchNormalization())
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=1))
    model_SVHC.add(Dropout(0.2))
    
    model_SVHC.add(Conv2D(128, (5, 5), padding='same'))
    model_SVHC.add(keras.layers.normalization.BatchNormalization())
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=2))
    model_SVHC.add(Dropout(0.2))
    
    
    model_SVHC.add(Conv2D(160, (5, 5), padding='same'))
    model_SVHC.add(keras.layers.normalization.BatchNormalization())
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=1))
    model_SVHC.add(Dropout(0.2))
    
    model_SVHC.add(Conv2D(192, (5, 5), padding='same'))
    model_SVHC.add(keras.layers.normalization.BatchNormalization())
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=2))
    model_SVHC.add(Dropout(0.2))
    
    model_SVHC.add(Conv2D(192, (5, 5), padding='same'))
    model_SVHC.add(keras.layers.normalization.BatchNormalization())
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=1))
    model_SVHC.add(Dropout(0.2))
    
    model_SVHC.add(Conv2D(192, (5, 5), padding='same'))
    model_SVHC.add(keras.layers.normalization.BatchNormalization())
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=2))
    model_SVHC.add(Dropout(0.2))
    
    
    model_SVHC.add(Conv2D(192, (5, 5), padding='same'))
    model_SVHC.add(keras.layers.normalization.BatchNormalization())
    model_SVHC.add(Activation('relu'))
    model_SVHC.add(MaxPooling2D(pool_size=(2, 2),padding='same',strides=1))
    model_SVHC.add(Dropout(0.2))
    
    model_SVHC.add(Flatten())
    
    model_SVHC.add(Dense(3072))
    model_SVHC.add(Activation('relu'))
    
    
    model_SVHC.add(Dropout(0.5))
    model_SVHC.add(Dense(pnumClasses))
    model_SVHC.add(Activation('softmax'))    
     
    return model_SVHC        

def evaluateModel(model,x_test,y_test,verbose):
    pLoss, pAcc = model.evaluate(x_test, y_test, verbose)
    print("Test Loss", pLoss)
    print("Test Accuracy", pAcc)
     
def RunMNIST(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    x_train, y_train, x_test, y_test = loadData()
    MNIST_model = model_MNIST()
    MNIST_sgd = optimizers.SGD(lr=learningRate, decay=weightDecay, momentum=momentum, nesterov=False)
    MNIST_model.compile(loss='categorical_crossentropy',optimizer=MNIST_sgd, metrics=['accuracy'])
    #print(MNIST_model.summary())
    #Training the model
    memT,cpuT,gpuT = YN.StartLogger("Keras","MNIST")
    start = time.time()
    MNIST_model.fit(x_train, y_train,batch_size=batchSize,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
    end = time.time()
    evaluateModel(MNIST_model,x_test, y_test, verbose=1)
    #Model performance evaluation
    YN.EndLogger(memT,cpuT,gpuT)
    print(str(datetime.timedelta(seconds=end-start)))

def RunCIFAR10(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    x_train, y_train, x_test, y_test = loadData()
    CIFAR_model = model_CIFAR10()
    CIFAR_sgd = optimizers.SGD(lr=learningRate, decay=weightDecay, momentum=momentum, nesterov=False)
    CIFAR_model.compile(loss='categorical_crossentropy',optimizer=CIFAR_sgd, metrics=['accuracy'])
    memT,cpuT,gpuT = YN.StartLogger("Keras","CIFAR10")
    start = time.time()
    CIFAR_model.fit(x_train, y_train,batch_size=batchSize,epochs=epochs,validation_data=(x_test, y_test),shuffle=True)
    end = time.time()
    evaluateModel(CIFAR_model,x_test, y_test, verbose=1)
    YN.EndLogger(memT,cpuT,gpuT)
    print(str(datetime.timedelta(seconds=end-start)))

def RunCIFAR100(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay):
    initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    x_train, y_train, x_test, y_test = loadData()
    CIFAR_model = model_CIFAR100()
    CIFAR_sgd = optimizers.SGD(lr=learningRate, decay=weightDecay, momentum=momentum, nesterov=False)
    CIFAR_model.compile(loss='categorical_crossentropy',optimizer=CIFAR_sgd, metrics=['accuracy'])
    memT,cpuT,gpuT = YN.StartLogger("Keras","CIFAR100")
    start = time.time()
    CIFAR_model.fit(x_train, y_train,batch_size=batchSize,epochs=epochs,validation_data=(x_test, y_test),shuffle=True)
    end = time.time()
    evaluateModel(CIFAR_model,x_test, y_test, verbose=1)
    YN.EndLogger(memT,cpuT,gpuT)
    print(str(datetime.timedelta(seconds=end-start)))

def RunSVHN(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay,fname):
    initParameters(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    x_train, y_train, x_test, y_test = loadDataSVHN(fname,True)
    SVHC_model = model_SVHC()
    SVHC_sgd = optimizers.SGD(lr=learningRate, decay=weightDecay, momentum=momentum, nesterov=False)
    SVHC_model.compile(loss='categorical_crossentropy',optimizer=SVHC_sgd, metrics=['accuracy'])
    memT,cpuT,gpuT = YN.StartLogger("Keras","SVHN")
    start = time.time()
    SVHC_model.fit(x_train, y_train,batch_size=batchSize,epochs=epochs,validation_data=(x_test, y_test),shuffle=True)
    end = time.time()
    evaluateModel(SVHC_model,x_test, y_test, verbose=1)
    YN.EndLogger(memT,cpuT,gpuT)
    print(str(datetime.timedelta(seconds=end-start)))
        
def runModel(dataset,batchSize=128,numClasses=10,epochs=12,learningRate=0.01,momentum=0.5,weightDecay=1e-6):
    
    if dataset is "mnist":
        RunMNIST(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    elif dataset is "cifar10":
        RunCIFAR10(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay)
    elif dataset is "cifar100":
        RunCIFAR100(dataset,batchSize,numClasses=100,epochs=epochs,learningRate=learningRate,momentum=momentum,weightDecay=weightDecay)
    elif dataset is "SVHN":
        fname = "./%s_32x32.mat"
        RunSVHN(dataset,batchSize,numClasses,epochs,learningRate,momentum,weightDecay,fname)
    else:
        print("Choose cifar10 or mnist or svhc")

In [ ]:
def main():
#    runModel("mnist",epochs=15)
#     runModel("cifar10",epochs=100)
     runModel("cifar100",epochs=200)
#     runModel("SVHN",epochs=100)

In [ ]:
main()